In [1]:
import pandas as pd

### Regression analysis

In [18]:
from src.data.additional_data import *
from src.utils.data_utils import *

age_path = 'data/raw/GeneralPopulationAge/DECENNIALDP2020.DP1-2024-11-14T134434.csv'
age_total, age_male, age_female = load_age_data(age_path, load_gender = True)
#print(age_total.head(5))


icpsr_path = 'data/raw/OPENICPSR/OPENICPSR_apparent_per_capita_alcohol_consumption.csv'
data_icpsr = load_icpsr(icpsr_path) # load + format data
print('available years: ', data_icpsr.keys())
print(data_icpsr['2016'].head(5))


path = "data/raw/IncomeBEA/"
filename = "GDP_PersonalIncome_USState.csv"
# DPI: disposable personal income
# PI: personal income
# PCE: personal consumption expenditures
# GDP: Gross Domestic Product
BEO_income_index = {
    'RealGDP': 1, 
    'RealPI':2, 
    'RealPCE': 3,
    'GDP': 4,  
    'PI': 5, 
    'DPI': 6,
    'PCE': 7,
    'RealPI/cap': 8,
    'RealPCE/cap': 9,
    'PI/cap': 10,
    'DPI/cap': 11,
    'PCE/cap': 12     
    #13 14 15 useless
}
BEA = loadBEA(path, filename, BEO_income_index)
BEA = get_mutliIndex_sub_df(BEA, ['RealPCE/cap', 'RealGDP', 'RealPI/cap'], ['2010']) 
BEA.index = BEA.index.droplevel(1)
print(BEA.head(5))


census_path = 'data/raw/Census/'
file_name_2020 = 'DECENNIALCD1182020.H2_rural_urban_US.csv'
file_name_2010 = 'DECENNIALCD1162010.H2_rural_urban_US.csv'


urban_2020_df = load_urban_frac_df(census_path, file_name_2020).reset_index()
urban_2010_df = load_urban_frac_df(census_path, file_name_2010).reset_index()
print(urban_2010_df.head(5))


available years:  dict_keys(['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'])
        state  ethanol_beer_gallons_per_capita  number_of_beers
0     Alabama                             1.09       258.370370
1      Alaska                             1.15       272.592593
2     Arizona                             1.14       270.222222
3    Arkansas                             0.93       220.444444
4  California                             0.96       227.555556
Income      RealPCE/cap    RealGDP  RealPI/cap
State                                         
Alabama         34044.0   199455.0     41757.0
Alaska          42794.0    53652.0     53121.0
Arizona         33721.0   282252.5     37162.0
Arkansas        33101.0   113000.9     40280.0
California      35036.0  2153852.8     43519.0
        state  total_pop  urban_pop  rural_pop  urban_frac
0     Alabama    2171853    1267224     904629    0.583476
1  

d:\Universitaet\Lausanne\Cours\MA3\ADA\Project\ada-2024-project-fandemondada\src\data\additional_data.py:85: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(',', '', regex=True)


In [28]:
# need to concatenate all data

merged = data_icpsr['2010'].merge(urban_2010_df, how='inner', on='state')

# BEA.index = BEA.index.droplevel(1)
merged = merged.merge(BEA, how='inner', left_on='state', right_on='State')
#merged = merged.merge(BEA, how='inner', on='state')
# merged = merged.merge(, how='inner', left_on='state', right_on='State')
merged = merged.rename(columns={"RealPCE/cap": "RealPCE_cap", "RealPI/cap": "RealPI_cap"}, errors="raise")


print(merged.columns)
print(merged.index)
print(merged.head(5))


Index(['state', 'ethanol_beer_gallons_per_capita', 'number_of_beers',
       'total_pop', 'urban_pop', 'rural_pop', 'urban_frac', 'RealPCE_cap',
       'RealGDP', 'RealPI_cap'],
      dtype='object')
RangeIndex(start=0, stop=40, step=1)
        state  ethanol_beer_gallons_per_capita  number_of_beers  total_pop  \
0     Alabama                             1.17       277.333333    2171853   
1      Alaska                             1.23       291.555556     306967   
2     Arizona                             1.23       291.555556    2844526   
3    Arkansas                             1.04       246.518519    1316299   
4  California                             0.97       229.925926   13680081   

   urban_pop  rural_pop  urban_frac  RealPCE_cap    RealGDP  RealPI_cap  
0    1267224     904629    0.583476      34044.0   199455.0     41757.0  
1     182989     123978    0.596119      42794.0    53652.0     53121.0  
2    2515504     329022    0.884332      33721.0   282252.5     37162.0 

In [ ]:
# need to add column of beer rating

In [ ]:
import statsmodels.formula.api as smf

ols_formula = 'rural_pop ~ total_pop*urban_frac + ethanol_beer_gallons_per_capita + number_of_beers + RealPCE_cap + RealGDP + RealPI_cap'
#ols_formula = 'review ~ urban_frac + ethanol_beer_gallons_per_capita + number_of_beers + RealPCE_cap + RealGDP + RealPI_cap'

mod = smf.ols(formula=ols_formula, data=merged)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              rural_pop   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.863e+26
Date:                Tue, 10 Dec 2024   Prob (F-statistic):               0.00
Time:                        13:59:07   Log-Likelihood:                 608.19
No. Observations:                  40   AIC:                            -1200.
Df Residuals:                      32   BIC:                            -1187.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 